In [73]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.metrics

#modelling
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [74]:
df=pd.read_csv(r'C:\Users\adity\PycharmProjects\mlprojects\notebook\data\raw.csv')
df.drop(columns=["MyUnknownColumn"],axis=1,inplace=True)

In [75]:
df.head()

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,MathScore,ReadingScore,WritingScore
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [76]:
x=df.drop(columns=["MathScore"],axis=1)

In [77]:
x.head()

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ReadingScore,WritingScore
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [78]:
y=df.MathScore

In [79]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: MathScore, dtype: int64

In [80]:
num_features=x.select_dtypes(exclude="object").columns
cat_features=x.select_dtypes(include="object").columns

from unicodedata import numeric
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScsler",numeric_transformer,num_features),

    ]
)


In [81]:
x=preprocessor.fit_transform(x)

In [82]:
x.shape

(30641, 19)

In [83]:
#seperate dataset into train trest split

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_train.shape,x_test.shape

((21448, 19), (9193, 19))

Create a Evaluate Function to give all metrics after model training

In [84]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [86]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoost Regressor" : CatBoostRegressor(verbose=False),
    "AdaBoost Regressor" : AdaBoostRegressor()

}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #make prediction
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)


    #Evaluate train and test dataset
    model_train_mae, model_train_rmse , model_train_r2 = evaluate_model(y_train,y_train_pred)

    model_test_mae, model_test_rmse , model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])


    print("Model performance for training set")
    print("- Root Mean Squared Error : {:.4f}".format(model_train_rmse))
    print("- Root Mean Absolute Error : {:.4f}".format(model_train_mae))
    print("- R2 score : {:.4f}".format(model_train_r2))



    print('----------------------------------------------------------')

    print("Model performance for test set")
    print("- Root Mean Squared Error : {:.4f}".format(model_test_rmse))
    print("- Root Mean Absolute Error : {:.4f}".format(model_test_mae))
    print("- R2 score : {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)



    print("="*35)
    print('\n')




Linear Regression
Model performance for training set
- Root Mean Squared Error : 5.3730
- Root Mean Absolute Error : 4.3067
- R2 score : 0.8756
----------------------------------------------------------
Model performance for test set
- Root Mean Squared Error : 5.3804
- Root Mean Absolute Error : 4.3042
- R2 score : 0.8738


Lasso
Model performance for training set
- Root Mean Squared Error : 6.6502
- Root Mean Absolute Error : 5.3337
- R2 score : 0.8094
----------------------------------------------------------
Model performance for test set
- Root Mean Squared Error : 6.6821
- Root Mean Absolute Error : 5.3591
- R2 score : 0.8054


Ridge
Model performance for training set
- Root Mean Squared Error : 5.3719
- Root Mean Absolute Error : 4.3060
- R2 score : 0.8756
----------------------------------------------------------
Model performance for test set
- Root Mean Squared Error : 5.3814
- Root Mean Absolute Error : 4.3055
- R2 score : 0.8738


K-Neighbors Regressor
Model performance for

In [87]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=["Model Name","R2_Score"]).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.873810
2,Ridge,0.873765
7,CatBoost Regressor,0.871114
6,XGBRegressor,0.864443
8,AdaBoost Regressor,0.845561
5,Random Forest Regressor,0.843612
3,K-Neighbors Regressor,0.843111
1,Lasso,0.805363
4,Decision Tree,0.735275
